In [ ]:
import sys
from IPython.display import Image, display
import sympy
sys.path.append('..')   # path for local package
import restflow
from restflow import symvec, symtools

# cKPZ equation

The cKPZ equation is given by:
\begin{equation*}
\partial_t\phi = -\kappa \nabla^4\phi + c_2 \nabla^2 |\nabla\phi|^2  + \eta,
\end{equation*}
with conserved noise. The quantity $\phi$ denotes the coarse-grained height field and describes the dynamics of the surface growth for conservative dynamics. For more on the physics of cKPZ, see Section I.2 from the accompanied manuscript.


We will only consider 1-loop graphs. For more loops, their contributions are of higher order in the perturbation series. 

Let's go back to the cKPZ case. The only relevant one-loop graph correcting the propagator for this example is:

In [ ]:
pil_img = Image(filename='./figures/kpz_graph.jpg')
display(pil_img)

The goal of this program is to translate these graphs into nested integrals and then to solve these integrals in the spirit of Wilson's shell renormalization to obtain the corrections of the vertex functions. Below are the steps to achieve this applied to the cKPZ model.

<b><ins>1st Step:</b></ins> Define the model parameters, the vectors, the propagators and the vertex functions:

In [ ]:
# define the model
class cKPZ:
    def __init__(self):
        self.alpha = 2
        self.c2, self.kap, self.D = sympy.symbols('c2 kappa D')

    def f(self,k):
        return self.kap*k**4

    def v2(self,k1,k2,q):
        return (self.c2*k1*k2*q**2,1)

# symbols for vectors
_q, _k, dot_kq = sympy.symbols('q k (k·q)')
# saves the vectors and dot products in a class
ctx = restflow.Context()
ctx.add_dot_product(_q,_k,dot_kq)
# create symbolic wave vectors
k = ctx.vector(_k)
q = ctx.vector(_q)

model = cKPZ()

<b><ins>2nd Step:</b></ins> Create the graph:

In [ ]:
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[2], angle=0.12)
v[0].link_vertex(v[1], 0.0)
v[1].link_vertex(v[2], 0.0)
v[1].add_outgoing(0.0)
g = restflow.Graph(v)

<b><ins>3rd Step:</b></ins> Label the graph by giving an input `labels` and using `label_edges`. 
(Optional): Visualize it using `plot_graph`. Create a LaTeX file for nicer rendering using `export_latex_graph`:

In [ ]:
labels = [k, q] # input array of sink vector and outgoing legs
g.label_edges(labels)
g.plot_graph()
# g.export_latex_graph('graph_plot')

<b><ins>4th Step:</b></ins> Convert graph into integral using ```convert```.

In [ ]:
# change of coordinates by renaming the labels
labels = [k, q] # input array of sink vector and outgoing legs
g.label_edges(labels)
expr = g.convert(model)
display(expr.num/expr.den)

<b><ins>5th Step:</b></ins> Calculate the integral using `integrate`:

In [ ]:
res = restflow.integrate([expr],5,labels)
display(res)

# Example 2: Graph with 2 external legs

We graphically corrected the propagator. To correct the 2-vertex, we need to consider all the graphs with 2 external legs. There are 2 1-loop graphs with 2 external legs for only 2-vertices:

In [ ]:
pil_img = Image(filename='./figures/kpz_2vertex.jpg')
display(pil_img)

We start with graph (A):

In [ ]:
pil_img = Image(filename='./figures/kpz_2vertex_graphb.jpg')
display(pil_img)

This graph actually represents 2 further graphs (figure (A1) and (A2)) depending on the labeling of the external legs. 

To set up the problem: Use the same system with before but we need to define the vector $p$ and the corresponding dot products:

In [ ]:
_p, dot_pk, dot_qp = sympy.symbols('p (k·p) (q·p)')
ctx.add_dot_product(_p,_k,dot_pk)
ctx.add_dot_product(_q,_p,dot_qp)
# create symbolic wave vectors
p = ctx.vector(_p)

Create the graph:

In [ ]:
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[3])
v[3].link_vertex(v[1])
v[2].add_outgoing()
v[3].add_outgoing()
g = restflow.Graph(v)
labels = [k, p, q-p]
g.label_edges(labels)


Use the method `convert_perm` to calculate all the permutations of the external momenta (figure (b) and (c)):

In [ ]:
exprs = g.convert_perm(model,labels)
for expr in exprs:
    display(expr.num/expr.den)

Use the function `integrate` to calculate the integrals and sum the two graphs:

In [ ]:
I_A = restflow.integrate(exprs,5,labels)
display(I_A)

We follow the same procedure for graph B:

In [ ]:
v = [restflow.Vertex() for i in range(4)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[1].link_vertex(v[3])
v[2].link_vertex(v[3])
v[1].add_outgoing()
v[2].add_outgoing()
g = restflow.Graph(v)
labels = [k, p, q-p]
exprs = g.convert_perm(model,labels)
I_B = restflow.integrate(exprs,5,labels)
display(I_B)

We add the graphical contributions to the 2-vertex from these graphs:

In [ ]:
I_2vertex = sympy.simplify(I_A+I_B)
display(I_2vertex)

The graphical correction for 1-loop graphs for the $c_2$ parameter for the cKPZ model is zero! This turns out to be true only at one-loop <sup>2</sup>.

# Flow equations

Once we have the corrections to the propagator and the 2-vertex, we can renormalize the model parameters by comparing the coefficients of the renormalized and unrenormalized vertex functions. Then, by proper scaling to restore the cut-off, the Wilson's flow equations can be found. 

In [ ]:
delta_l = sympy.symbols('δl')
psi_kappa = sympy.simplify((sympy.simplify(-symtools.all_coeffs(res/q**2/delta_l,[q.sym])[q.sym**2])).as_expr()/model.kap)
display(psi_kappa)

Note that $\psi_D=0$ since the only possible diagram is proportional to $k^4$ while the original spectrum of noise is proportional to $k^2$. Thus, this correction corresponds to higher derivatives and thus irrelevant. 

We now define the dimensionless parameter
\begin{equation}
\bar{c}_2 \equiv \frac{c_2 D^{1/2}}{\kappa^{3/2}}K_d^{1/2}\Lambda^{d/2-1}.
\end{equation}

Express $\psi_\kappa$ wrt $\bar{c}_2$:

In [ ]:
d, Kd, Lambda, c2bar = sympy.symbols('d K_d Lambda c2_b')

psi_kappa = sympy.simplify(psi_kappa.subs(model.c2, c2bar*model.kap**sympy.Rational(3,2)*model.D**sympy.Rational(-1,2)*Kd**sympy.Rational(-1,2)*Lambda**(-d/2+1)))
display(psi_kappa)

Thus the graphical corrections are $\psi_D=\psi_2=0$ and $\psi_\kappa = \frac{2\bar{c}_2^2}{d}$.

Using dimensional arguments we can now obtain the flow equation of $\bar{c}_2$:
\begin{equation}
\partial_l \bar{c}_2 = \left(-\frac{d-2}{2}-\frac{3}{2}\psi_\kappa\right)\bar{c}_2 = \left(-\frac{d-2}{2}-\frac{3\bar{c}_2^2}{d}\right)\bar{c}_2
\end{equation}

From this it is straightforward to obtain the non-trivial fixed points which appear only for $d<2$. Specifically, we get for $\epsilon=2-d$

In [ ]:
epsilon = sympy.symbols('epsilon')
flow_c2 = (-(d-2)/2-3*c2bar**2/d)*c2bar
sol = sympy.solve(sympy.series(flow_c2,epsilon,0,2),c2bar)
sol = [sympy.series(element.subs(d,2-epsilon),epsilon,0,1).removeO() for element in sol]
for element in sol:
  display(element)

# References

<sup>1</sup> Sun, Tao, Hong Guo, and Martin Grant. "Dynamics of driven interfaces with a conservation law." Physical Review A 40.11 (1989): 6763.

<sup>2</sup> H. K. Janssen, “On Critical Exponents and the Renormalization of the Coupling Constant in Growth Models with Surface Diffusion,” Phys. Rev. Lett. 78, 1082–108 (1997).
